In [19]:
import cv2
import os
from PIL import Image
import sys
import numpy as np
import matplotlib.pyplot as plt
from skimage import measure, morphology
from skimage.color import label2rgb
from skimage.measure import regionprops
import re 

In [20]:
# read the input image
imgi=os.path.join('/home/administrator/TE/in2.jpg')
img = cv2.imread(imgi)
img = cv2.threshold(img, 127, 255, cv2.THRESH_BINARY)[1]  # ensure binary

# connected component analysis by scikit-learn framework
blobs = img > img.mean()
blobs_labels = measure.label(blobs, background=1)

In [21]:
#variable dicleration 
the_biggest_component = 0
total_area = 0
counter = 0
average = 0.0
for region in regionprops(blobs_labels):
    if (region.area > 10):
        total_area = total_area + region.area
        counter = counter + 1
    # print region.area # (for debugging)
    # take regions with large enough areas
    if (region.area >= 250):
        if (region.area > the_biggest_component):
            the_biggest_component = region.area

average = (total_area/counter)
print("the_biggest_component: " + str(the_biggest_component))
print("average: " + str(average))

# experimental-based ratio calculation
# a4_constant is used as a threshold value to remove connected pixels
# are smaller than a4_constant for A4 size scanned documents
a4_constant = ((average/84.0)*250.0)+100
print("a4_constant: " + str(a4_constant))

# remove the connected pixels are smaller than a4_constant
b = morphology.remove_small_objects(blobs_labels, a4_constant)
# save the the pre-version which is the image is labelled with colors
# as considering connected components
b = b.astype(np.uint8)
plt.imsave('pre_version.png', b)
# read the pre-version
img = cv2.imread('pre_version.png', 0)
# ensure binary
img = cv2.threshold(img, 0, 255, cv2.THRESH_BINARY_INV | cv2.THRESH_OTSU)[1]
# save the the result which is having all the signatures in the image.
cv2.imwrite("output.jpg", img)

the_biggest_component: 4932
average: 101.85866666666666
a4_constant: 403.1507936507937


True

In [22]:
image = cv2.imread('/home/administrator/TE/output.jpg')
#--- Image was too big hence I resized it ---
image = cv2.resize(image, (0, 0), fx = 0.5, fy = 0.5)

#--- Converting image to grayscale ---
gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

#--- Performing inverted binary threshold ---
retval, thresh_gray = cv2.threshold(gray, 0, 255, type = cv2.THRESH_BINARY_INV | cv2.THRESH_OTSU)
#--- finding contours ---
contours, hierarchy = cv2.findContours(thresh_gray,cv2.RETR_EXTERNAL, \
                                              cv2.CHAIN_APPROX_SIMPLE)

for i, c in enumerate(contours):
    if cv2.contourArea(c) > 100:
        x, y, w, h = cv2.boundingRect(c)
        roi = image[y  :y + h, x : x + w ]
        cv2.imwrite('sign_{}.jpg'.format(i), roi)
cv2.destroyAllWindows()